In [ ]:
import pickle
import numpy as np

with open('/Users/owenburns/workareas/cavrel-platooning/missions/datapoints/py3_1_parking_loop_21_58_14.pkl', 'rb') as f:
    data = pickle.load(f)#, encoding='latin1') # latin1 is essential for loading python2 pickle files. Not necessary for python3 files

cars = [np.array(car) for car in zip(*data)]

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm
from matplotlib.colors import LinearSegmentedColormap

def plot_gradient_lines(xs, ys, colors, label=None):
    # Prepare points for line collection
    points = np.array([xs, ys]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    
    # Create a continuous linear colormap
    cmap = LinearSegmentedColormap.from_list("mycmap", colors)

    # Setting up the LineCollection
    lc = LineCollection(segments, cmap=cmap, linewidth=3)
    # Set the values used for colormapping
    lc.set_array(np.linspace(0, 1, len(xs)))
    lc.set_label(label)

    # Adding the collection to the plot
    plt.gca().add_collection(lc)

xs1 = cars[0][:, 0]
ys1 = cars[0][:, 1]

xs2 = cars[1][:, 0]
ys2 = cars[1][:, 1]

# xs3 = cars[2][:, 0]
# ys3 = cars[2][:, 1]

plt.figure(figsize=(10, 6))
# fig, ax = plt.subplots(2, 1, figsize=(10, 14))

# Plot each line with gradients
plot_gradient_lines(xs1, ys1, ["blue", "red"], label='target vehicle')
plot_gradient_lines(xs2, ys2, ["yellow", "orange"], label='platoon 1')
# plot_gradient_lines(xs3, ys3, ["green", "purple"], label='platoon 2')
plt.xlim(min(xs1.min(), xs2.min()), max(xs1.max(), xs2.max()))
plt.ylim(min(ys1.min(), ys2.min()), max(ys1.max(), ys2.max()))

plt.legend()
plt.title('Gradient Lines Representing Motion Over Time')
plt.xlabel('X')
plt.ylabel('Y')
plt.grid(True)
plt.show()

In [ ]:
print(len(cars[0]))
print(len(cars[1]))
with open("/Users/owenburns/workareas/cavrel-platooning/missions/datapoints/py3_1_parking_loop_21_58_14_LINE.pkl", 'rb') as l:
    lines = pickle.load(l)
print(len(lines))

In [ ]:
print(71-17)

In [ ]:
from typing import List
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.animation import FuncAnimation
from matplotlib.text import Annotation
from matplotlib.lines import Line2D
from IPython.display import HTML
import matplotlib
import math
import json
import numpy as np
matplotlib.rcParams['animation.embed_limit'] = 2**128
EARTH_RADIUS = 6371e3
DUE_EAST = 90
BROADCAST_INTERVAL = 0.1

with open("/Users/owenburns/workareas/cavrel-platooning/missions/tracks/garage_loop.json", 'r') as f:
    track = json.load(f)

with open("/Users/owenburns/workareas/cavrel-platooning/missions/datapoints/py3_1_parking_loop_21_58_14_LINE.pkl", 'rb') as l:
    lines = pickle.load(l)

with open("/Users/owenburns/workareas/cavrel-platooning/missions/datapoints/py3_1_parking_loop_21_58_14_CLOSEST.pkl", 'rb') as p:
    closest_points = pickle.load(p)

center_latitude = track['center']['lat']
center_longitude = track['center']['lon']
center_orientation = DUE_EAST

def coords_to_local(target_lat, target_lon):
    # Convert lat, lon to radians
    target_lat_rad, target_lon_rad = math.radians(target_lat), math.radians(target_lon)
    current_lat_rad, current_lon_rad = math.radians(center_latitude), math.radians(center_longitude)

    x = EARTH_RADIUS * (target_lon_rad - current_lon_rad) * math.cos((current_lat_rad + target_lat_rad) / 2)
    y = EARTH_RADIUS * (target_lat_rad - current_lat_rad)

    angle = math.radians(center_orientation - DUE_EAST)
    qx = math.cos(angle) * x - math.sin(angle) * y
    qy = math.sin(angle) * x + math.cos(angle) * y
    return qx, qy

def heading_velocity_to_components(heading, velocity):
    # Convert heading (degrees) to radians
    heading_rad = math.radians(heading)
    # Calculate the x and y components of the velocity
    vx = velocity * math.sin(heading_rad)
    vy = velocity * math.cos(heading_rad)
    return vx, vy

def vector_to_arrow(vector):
    x, y = vector[0], vector[1]
    vx, vy = heading_velocity_to_components(vector[2], vector[3])
    return (x, y, vx, vy)

# set up the plot and plot the runing track
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(autoscale_on=False, xlim=(-100, 100), ylim=(-100, 100))
ax.plot(xs1, ys1, 'k-', label='Track Path')  # green dots connected by lines
ax.plot(xs1[0], ys1[0], 'yo', label='Start/End Point')
ax.set_aspect('equal')

ego_vectors = np.array(cars[1])
target_vectors = np.array(cars[0])

to_delete = np.random.choice(np.arange(ego_vectors.shape[0]), size=54, replace=False)

ego_vectors = np.delete(ego_vectors, to_delete, axis=0)
print(ego_vectors.shape)
target_vectors = np.delete(target_vectors, to_delete, axis=0)

ex, ey, edx, edy = vector_to_arrow(ego_vectors[0])
# v, h = ego_targets[0]
# vacc, hacc = ego_controls[0]
tx, ty, tdx, tdy = vector_to_arrow(target_vectors[0])
xsl, ysl = lines[0] # x0_opt, y0_opt, dx_opt, dy_opt = lines[0]
closest = closest_points[0]
ego_arrow = patches.Arrow(ex, ey, edx, edy, width=2, fc='red', ec='red')
target_arrow = patches.Arrow(tx, ty, tdx, tdy, width=2, fc='blue', ec='blue')
ego_text = Annotation(f"dx: {edx:.2f}, dy: {edy:.2f}", xy=(ex, ey), xytext=(ex, ey))#\nv: {v}, h: {h}
# target_text = Annotation(f"dx: {tdx:.2f}, dy: {tdy:.2f}", xy=(tx, ty), xytext=(tx, ty))
line = Line2D(xsl, ysl, color='green') # line = Line2D([x0_opt, closest[0]], [y0_opt, closest[1]], color='green')
pt = Line2D([closest[0]], [closest[1]], marker='o', color='green') # pt = Line2D([closest[0]], [closest[1]], marker='o', color='green')


def init_func() -> list[patches.Arrow]:
    print('init_func')
    ax.add_patch(ego_arrow)
    ax.add_patch(target_arrow)
    ax.add_artist(ego_text)
    # ax.add_artist(target_text)
    ax.add_artist(line)
    ax.add_artist(pt)
    return ego_arrow, target_arrow, ego_text#, target_text

def func(frame, *fargs) -> list[patches.Arrow]:
    global ego_arrow, target_arrow, ego_text, pt#, target_text, line, pt

    ax.patches[ax.patches.index(ego_arrow)].remove()
    ax.patches[ax.patches.index(target_arrow)].remove()

    _, _, e_heading, _ = ego_vectors[frame]
    _, _, t_heading, _ = target_vectors[frame]
    # v, h = ego_targets[frame]
    ex, ey, edx, edy = vector_to_arrow(ego_vectors[frame])
    tx, ty, tdx, tdy = vector_to_arrow(target_vectors[frame])
    ego_arrow = patches.Arrow(ex, ey, edx*BROADCAST_INTERVAL, edy*BROADCAST_INTERVAL, width=2, fc='red', ec='red')
    target_arrow = patches.Arrow(tx, ty, tdx*BROADCAST_INTERVAL, tdy*BROADCAST_INTERVAL, width=2, fc='blue', ec='blue')
    ego_text.set_text(f"dx: {edx*BROADCAST_INTERVAL:.2f}, dy: {edy*BROADCAST_INTERVAL:.2f}, heading: {e_heading:.2f}")#\nv: {v:.2f}, t_h: {h:.2f}")#\naccx: {accx:.2f}, accy: {accy:.2f}")
    ego_text.set_position((ex, ey))
    # target_text.set_text(f"dx: {tdx*broadcast_int:.2f}, dy: {tdy*broadcast_int:.2f}, heading: {t_heading:.2f}")
    # target_text.set_position((tx, ty))
    ax.add_patch(ego_arrow)
    ax.add_patch(target_arrow)

    xsl, ysl = lines[frame]# x0_opt, y0_opt, dx_opt, dy_opt = lines[frame]
    closest = closest_points[frame]
    line.set_data(xsl, ysl) # line.set_data([x0_opt, closest[0]], [y0_opt, closest[1]])
    pt.set_data([closest[0]], [closest[1]])

    return ego_arrow, target_arrow, ego_text#, target_text

ax.set_xlabel('Meters East of Center')
ax.set_ylabel('Meters North of Center')
ax.set_title('Vehicle Path and Vectors')
ax.legend()
ax.grid()

# Create animation
ani = FuncAnimation(fig, func, range(min(len(ego_vectors), len(target_vectors))), init_func, blit=True, interval=50) # Adjust interval as needed
HTML(ani.to_jshtml())

# TODO: fix issue with cubic spline controller that the car follows the point next to it then the line is close to
# vertical and thus swoops around. do this by constraining the distance over which the spline is searchable to the maximum distance
# the ego vehicle is away from a platoon vehicle

In [ ]:
print(cars[0].shape)

In [ ]:
# plot all car velocities over time
time = np.linspace(0, 0.1*len(xs1), len(xs1))
plt.figure(figsize=(10, 6))

car0_v = cars[0][:,3]
car1_v = cars[1][:,3]
# car2_v = cars[2][:,3]

plt.plot(time, car0_v, label='target vehicle')
plt.plot(time, car1_v, label='platoon 1')
# plt.plot(time, car2_v, label='platoon 2')
plt.legend()
plt.title('Velocity Over Time')
plt.xlabel('Time (s)')
plt.ylabel('Velocity (m/s)')
plt.grid(True)
plt.show()

In [ ]:
# plot all car headings over time
time = np.linspace(0, 0.1*len(xs1), len(xs1))
plt.figure(figsize=(10, 6))

car0_v = cars[0][:,2]
car1_v = cars[1][:,2]
# car2_v = cars[2][:,3]

plt.plot(time, car0_v, label='target vehicle')
plt.plot(time, car1_v, label='platoon 1')
# plt.plot(time, car2_v, label='platoon 2')
plt.legend()
plt.title('heading Over Time')
plt.xlabel('Time (s)')
plt.ylabel('heading')
plt.grid(True)
plt.show()

In [ ]:
# plot the difference between max and min velocity over time
time = np.linspace(0, 0.1*len(xs1), len(xs1))
plt.figure(figsize=(10, 6))

car0_v = np.expand_dims(cars[0][:,3], 1)
car1_v = np.expand_dims(cars[1][:,3], 1)
car2_v = np.expand_dims(cars[2][:,3], 1)

concat_speeds = np.concatenate((car0_v, car1_v, car2_v), axis=1)
max_speeds = np.max(concat_speeds, axis=1)
min_speeds = np.min(concat_speeds, axis=1)
speed_diff = max_speeds - min_speeds

plt.plot(time, speed_diff, label='speed difference')
plt.legend()
plt.title('Velocity Over Time')
plt.xlabel('Time (s)')
plt.ylabel('Velocity (m/s)')
plt.grid(True)
plt.show()

In [ ]:
errors = np.array([])
max_errors = np.zeros((cars[0].shape[0], 1))

CAR_LENGTH = 0.779
FOLLOW_DISTANCE = 2.0

def distance_to_line(x0, y0, dx, dy, x, y):
    lambda_val = ((x - x0) * dx + (y - y0) * dy) / (dx**2 + dy**2)
    closest_point = np.array([x0 + lambda_val * dx, y0 + lambda_val * dy])
    distance = np.linalg.norm(closest_point - np.array([x, y]))
    return distance, closest_point

for i, car in enumerate(cars):
    if i == 0:
        continue
    
    current_x = cars[i][:, 0]
    current_y = cars[i][:, 1]

    for j in range(0, i):
        position = i - j
        goal_follow_distance = FOLLOW_DISTANCE*position + CAR_LENGTH*(position - 1)
        
        # x_goal = cars[j][:, 0] - goal_follow_distance*np.sin(cars[j][:, 2])
        # y_goal = cars[j][:, 1] - goal_follow_distance*np.cos(cars[j][:, 2])

        # error = np.sqrt((current_x - x_goal)**2 + (current_y - y_goal)**2)

        # Reference car position and heading
        ref_x = cars[j][:, 0]
        ref_y = cars[j][:, 1]
        ref_heading = cars[j][:, 2]

        # Project the current car's position onto the line defined by the reference car's position and heading
        dx = current_x - ref_x
        dy = current_y - ref_y
        
        # Compute the projection of the vector difference onto the direction of the reference car
        projected_length = dx * np.cos(ref_heading) + dy * np.sin(ref_heading)
        
        # Calculate the error as the absolute difference between the projected length and the goal follow distance
        error = np.abs(projected_length - goal_follow_distance)

        errors = np.append(errors, error)

        error = np.expand_dims(error, 1)
        max_errors = np.concatenate((max_errors, error), axis=1)
        max_errors = np.max(max_errors, axis=1)
        max_errors = np.expand_dims(max_errors, 1)

print(errors.shape)

print(f"95th percentile error: {np.percentile(errors, 95)}")

time = np.linspace(0, 0.1*len(xs1), len(xs1))
plt.figure(figsize=(10, 6))

plt.plot(time, max_errors, label='max error')
plt.legend()
plt.title('Error Over Time')
plt.xlabel('Time (s)')
plt.ylabel('Error (m)')
plt.grid(True)
plt.show()
